In [ ]:
!pip install sec-api

In [6]:
from sec_api import ExtractorApi
import pandas as pd

In [3]:
def get_section(filing_url, section = '1'):
    res = filing_url[:filing_url.index("htm") + len("htm")]
    section_text = extractorApi.get_section(str(res), section, "html")
    return section_text

def read_and_clean_table(text):
    tables = pd.read_html(text)
    cleaned_tables = []
    for table in tables:
        table = table.dropna(how='all')
        drop_cols = []
        for col in table.columns:
            nan_cnt = table[col].isnull().sum()
            if nan_cnt >= table.shape[0]/2:
                drop_cols.append(col)
        table = table.drop(labels=drop_cols, axis=1)
        cleaned_tables.append(table)
    return cleaned_tables

def table_to_str(table):
    table = table.values.tolist()
    table_str = []
    for row in table:
        row_cl = [str(data).replace('\xa0', ' ').replace('\\x', ' ').replace('(',' ').replace(')',' ') for data in row]
        table_str.append(' | '.join(row_cl))
    table_str = ' \n '.join(table_str)
    return table_str

In [4]:
api_key = '8aa664eee7896b98038d05ca58f3779d3d9dfa9ca28ad1394702477226556894'
extractorApi = ExtractorApi(api_key)

In [ ]:
url = "https://www.sec.gov/Archives/edgar/data/0001408710/000119312517265630/d396247d10k.htm"
section = '6'
tables = []
for table in read_and_clean_table('<html><body>' + get_section(url, section) + '</body></html>'):
    table_data = table_to_str(table)
    tables.append(table_data)
print(tables)